In [1]:
from preprocess import *
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, LSTM, Activation
from keras.utils import to_categorical
import wandb
from wandb.keras import WandbCallback
import matplotlib.pyplot as plt
import sklearn.metrics as metrics

Using TensorFlow backend.


In [2]:
wandb.init()
config = wandb.config

config.max_len = 21
config.buckets = 50

# Save data to array file first
save_data_to_array(max_len=config.max_len, n_mfcc=config.buckets)

#labels=np.array(["chirping_birds", "crickets", "crow", 
#                 "frog", "insects"])
labels=np.array(["GOC", "GRA", "GST", 
                 "GWG", "GWC"])

Saving vectors of label - 'GWG': 100%|█████████████████████████████████████████████████| 52/52 [00:01<00:00, 37.26it/s]


In [3]:
# Loading train/test set
X_train, X_test, X_val, y_train, y_test, y_val = get_train_test()

In [4]:
# Setting channels to 1 to generalize stereo sound to 1 channel
channels = 1
config.epochs = 50
config.batch_size = 100

# Number of classes
num_classes = 5

# Reshape X_train and X_test to include a 4th dimension (channels)
X_train = X_train.reshape(X_train.shape[0], config.buckets, config.max_len, channels)
X_test = X_test.reshape(X_test.shape[0], config.buckets, config.max_len, channels)
X_val = X_val.reshape(X_val.shape[0], config.buckets, config.max_len, channels)

In [5]:
# Spectrogram visualized of 0th element
print(X_train.shape)
#plt.imshow(X_train[500, :, :, 0])

(153, 50, 21, 1)


In [6]:
# Getting vector number where each number corresponds to a label
y_train_hot = to_categorical(y_train)
y_test_hot = to_categorical(y_test)
y_val_hot = to_categorical(y_val)

In [7]:
# Building the model
model = Sequential()

input_shape= (config.buckets, config.max_len, channels)

model.add(Conv2D(24, (3, 3), strides=(1, 1), input_shape=input_shape))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))
model.add(Activation('relu'))

model.add(Conv2D(48, (3, 3), padding="valid"))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))
model.add(Activation('relu'))

model.add(Conv2D(48, (3, 1), padding="valid"))
model.add(Activation('relu'))

model.add(Flatten())
model.add(Dropout(rate=0.5))

model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(rate=0.5))

model.add(Dense(len(labels)))
model.add(Activation('softmax'))
model.summary()
# Conv2D: 
#    Filters: 32
#    Kernel_size: (3,3) (height/width of the 2D convolution window)     
'''model.add(Conv2D(32, (3, 3),
    input_shape=(config.buckets, config.max_len, channels),
    activation='relu'))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))'''


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 48, 19, 24)        240       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 24, 9, 24)         0         
_________________________________________________________________
activation_1 (Activation)    (None, 24, 9, 24)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 22, 7, 48)         10416     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 11, 3, 48)         0         
_________________________________________________________________
activation_2 (Activation)    (None, 11, 3, 48)         0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 9, 3, 48)        

"model.add(Conv2D(32, (3, 3),\n    input_shape=(config.buckets, config.max_len, channels),\n    activation='relu'))\n\nmodel.add(MaxPooling2D(pool_size=(2, 2)))\n\nmodel.add(Flatten())\n\nmodel.add(Dense(128, activation='relu'))\nmodel.add(Dense(num_classes, activation='softmax'))"

In [8]:
# Configure CNN for training
model.compile(loss="categorical_crossentropy",
                  optimizer="adam",
                  metrics=['accuracy'])

In [9]:
wandb.init()
print(y_train_hot.shape)
print(labels.shape)
print(X_train.shape)
# Train the CNN model
#    X_train: Input data
#    y_train_hot: Target data
model.fit(X_train, y_train_hot, epochs=config.epochs, validation_data=(X_val, y_val_hot), callbacks=[WandbCallback(data_type="image", labels=labels)])

(153, 5)
(5,)
(153, 50, 21, 1)

Train on 153 samples, validate on 103 samples
Epoch 1/50
153/153 [==============================] - ETA: 4s - loss: 6.3866 - accuracy: 0.25 - ETA: 1s - loss: 5.2013 - accuracy: 0.32 - ETA: 0s - loss: 4.6251 - accuracy: 0.32 - 1s 10ms/step - loss: 3.9702 - accuracy: 0.3137 - val_loss: 1.6140 - val_accuracy: 0.4466
Epoch 2/50
153/153 [==============================] - ETA: 0s - loss: 2.3227 - accuracy: 0.37 - ETA: 0s - loss: 2.1149 - accuracy: 0.35 - ETA: 0s - loss: 2.0037 - accuracy: 0.36 - 0s 2ms/step - loss: 2.0069 - accuracy: 0.3529 - val_loss: 1.3922 - val_accuracy: 0.4078
Epoch 3/50
153/153 [==============================] - ETA: 0s - loss: 1.5084 - accuracy: 0.43 - ETA: 0s - loss: 1.4560 - accuracy: 0.45 - ETA: 0s - loss: 1.4355 - accuracy: 0.46 - 0s 2ms/step - loss: 1.4195 - accuracy: 0.4837 - val_loss: 1.3732 - val_accuracy: 0.4078
Epoch 4/50
153/153 [==============================] - ETA: 0s - loss: 1.4537 - accuracy: 0.43 - ETA: 0s - loss: 1.350

153/153 [==============================] - ETA: 0s - loss: 0.4869 - accuracy: 0.87 - ETA: 0s - loss: 0.6072 - accuracy: 0.79 - ETA: 0s - loss: 0.5735 - accuracy: 0.81 - ETA: 0s - loss: 0.5855 - accuracy: 0.80 - 0s 3ms/step - loss: 0.6218 - accuracy: 0.7843 - val_loss: 0.8452 - val_accuracy: 0.6214
Epoch 31/50
153/153 [==============================] - ETA: 0s - loss: 0.3788 - accuracy: 0.84 - ETA: 0s - loss: 0.4928 - accuracy: 0.82 - ETA: 0s - loss: 0.4840 - accuracy: 0.82 - 0s 2ms/step - loss: 0.4901 - accuracy: 0.8301 - val_loss: 0.8403 - val_accuracy: 0.6505
Epoch 32/50
153/153 [==============================] - ETA: 0s - loss: 0.5543 - accuracy: 0.78 - ETA: 0s - loss: 0.4730 - accuracy: 0.81 - ETA: 0s - loss: 0.5190 - accuracy: 0.79 - ETA: 0s - loss: 0.5050 - accuracy: 0.76 - 0s 2ms/step - loss: 0.5178 - accuracy: 0.7582 - val_loss: 0.8160 - val_accuracy: 0.6505
Epoch 33/50
153/153 [==============================] - ETA: 0s - loss: 0.4588 - accuracy: 0.87 - ETA: 0s - loss: 0.5673 -

In [10]:
# Save the keras model
model.save("geo_cnn_model.h5")
print("Model has been saved.")

Model has been saved.


## Running the IntelliChirp Biophony CNN

In [11]:
from keras.models import load_model

# Load the model
loaded_model = load_model('ant_cnn_model.h5')

In [12]:
# Summarize the model
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 48, 19, 24)        240       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 24, 9, 24)         0         
_________________________________________________________________
activation_1 (Activation)    (None, 24, 9, 24)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 22, 7, 48)         10416     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 11, 3, 48)         0         
_________________________________________________________________
activation_2 (Activation)    (None, 11, 3, 48)         0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 9, 3, 48)         

In [13]:
y_pred_ohe = loaded_model.predict(X_test)  # shape=(n_samples, 12)
y_pred_labels = np.argmax(y_pred_ohe, axis=1)  # only necessary if output has one-hot-encoding, shape=(n_samples)

confusion_matrix = metrics.confusion_matrix(y_true=y_test, y_pred=y_pred_labels)  # shape
print(confusion_matrix)

for class_i in range(len(labels)) :
    indices = np.argwhere(y_test == class_i)
    sum = 0
    for index in indices:
        sum += (y_test[index] == y_pred_labels[index])
    if(len(indices) > 0) : mean = sum/len(indices)
    else : mean = "N/A"
    print("Accuracy for class", labels[class_i], ":", mean)

print("Overall Accuracy :", np.mean(y_test == y_pred_labels))

[[ 4  0  0  0  0]
 [ 0 21  0  5  0]
 [ 0  2  0  1  0]
 [ 0  9  0 10  4]
 [ 0  2  0  2  4]]
Accuracy for class GOC : [1.]
Accuracy for class GRA : [0.80769231]
Accuracy for class GST : [0.]
Accuracy for class GWG : [0.43478261]
Accuracy for class GWC : [0.5]
Overall Accuracy : 0.609375


In [15]:
## Running the model

n_mfcc = config.buckets
max_len = config.max_len
# convert file to wav2mfcc
# Mel-frequency cepstral coefficients
file_path = "./prediction/nature_sc.wav"
big_wave, sr = librosa.load(file_path, mono=True, sr=None)
#print(wave.shape, sr)

classification = []

for sec_index in range( int(big_wave.shape[0] / sr) ) :
    start_sec = sec_index
    end_sec = sec_index + 1
    
    sec_to_trim = np.array( [ float(start_sec), float(end_sec) ] )
    print(sec_to_trim)
    sec_to_trim = np.ceil( sec_to_trim * sr )

    wave = big_wave[int(sec_to_trim[0]) : int(sec_to_trim[1])]
    print(wave)

    wave = np.asfortranarray(wave[::3])
    mfcc = librosa.feature.mfcc(wave, sr=16000, n_mfcc=n_mfcc)

    # If maximum length exceeds mfcc lengths then pad the remaining ones
    if (max_len > mfcc.shape[1]):
        pad_width = max_len - mfcc.shape[1]
        mfcc = np.pad(mfcc, pad_width=((0, 0), (0, pad_width)), mode='constant')

    # Else cutoff the remaining parts
    else:
        mfcc = mfcc[:, :max_len]

    # Convert wav to MFCC
    prediction_data = wav2mfcc('./prediction/nature_sc.wav')
    prediction_data = mfcc
    print(prediction_data.shape)
    #print(wav2mfcc())
    # Reshape to 4 dimensions
    prediction_data = prediction_data.reshape(1, config.buckets, config.max_len, channels)
    #prediction_data = prediction_data.reshape(1, 20, config.max_len, channels)

    # Run the model on the inputted file
    predicted = loaded_model.predict(prediction_data)

    # Output the prediction values for each class
    print ('PREDICTED VALUES')
    labels_indices = range(len(labels))
    max_value = 0
    max_value_index = 0
    for index in labels_indices:
        print('\n', labels[index], ": ", '%.08f' % predicted[0,index])
        if predicted[0,index] > max_value:
            max_value_index = index
            max_value = predicted[0,index]

    # Output the prediction
    if max_value < 0.5:
        print("GUESS: Nothing")
        classification.append( { "class" : "Nothing", "timestamp" : start_sec } )
    else:
        print('\n\nGUESS: ', labels[max_value_index])
        classification.append( { "class" : labels[max_value_index], "timestamp" : start_sec } )

print(classification)

[0. 1.]
[ 0.0000000e+00  1.5258789e-05  0.0000000e+00 ...  3.3020020e-02
  1.2680054e-02 -8.7432861e-03]
(50, 21)
PREDICTED VALUES

 GOC :  0.00110971

 GRA :  0.00687433

 GST :  0.00060519

 GWG :  0.28046575

 GWC :  0.71094495


GUESS:  GWC
[1. 2.]
[-0.03717041 -0.05769348 -0.06455994 ...  0.01766968  0.01895142
  0.01779175]
(50, 21)
PREDICTED VALUES

 GOC :  0.00000150

 GRA :  0.66413796

 GST :  0.00009404

 GWG :  0.00712661

 GWC :  0.32863984


GUESS:  GRA
[2. 3.]
[ 0.02345276  0.02101135  0.01712036 ... -0.01161194 -0.0141449
 -0.01431274]
(50, 21)
PREDICTED VALUES

 GOC :  0.00008299

 GRA :  0.22556210

 GST :  0.00302639

 GWG :  0.10296817

 GWC :  0.66836035


GUESS:  GWC
[3. 4.]
[-0.01583862 -0.01066589 -0.00762939 ... -0.0377655  -0.03556824
 -0.02685547]
(50, 21)
PREDICTED VALUES

 GOC :  0.00076802

 GRA :  0.19427927

 GST :  0.00463253

 GWG :  0.03950845

 GWC :  0.76081169


GUESS:  GWC
[4. 5.]
[-0.02836609 -0.02510071 -0.02012634 ...  0.0138855  -0.00386047
 -

 GWG :  0.40232626

 GWC :  0.34449333
GUESS: Nothing
[{'class': 'GWC', 'timestamp': 0}, {'class': 'GRA', 'timestamp': 1}, {'class': 'GWC', 'timestamp': 2}, {'class': 'GWC', 'timestamp': 3}, {'class': 'Nothing', 'timestamp': 4}, {'class': 'GRA', 'timestamp': 5}, {'class': 'GWC', 'timestamp': 6}, {'class': 'GRA', 'timestamp': 7}, {'class': 'GRA', 'timestamp': 8}, {'class': 'GWC', 'timestamp': 9}, {'class': 'GRA', 'timestamp': 10}, {'class': 'GWG', 'timestamp': 11}, {'class': 'Nothing', 'timestamp': 12}, {'class': 'GWG', 'timestamp': 13}, {'class': 'Nothing', 'timestamp': 14}, {'class': 'GWC', 'timestamp': 15}, {'class': 'GRA', 'timestamp': 16}, {'class': 'GRA', 'timestamp': 17}, {'class': 'GRA', 'timestamp': 18}, {'class': 'GRA', 'timestamp': 19}, {'class': 'GRA', 'timestamp': 20}, {'class': 'GRA', 'timestamp': 21}, {'class': 'GRA', 'timestamp': 22}, {'class': 'GRA', 'timestamp': 23}, {'class': 'GRA', 'timestamp': 24}, {'class': 'GRA', 'timestamp': 25}, {'class': 'Nothing', 'timestamp':